### Importing the Libraries

In [ ]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import LSTM
from keras import models, layers
from keras_visualizer import visualizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score
from sklearn.metrics import f1_score, accuracy_score ,specificity_score, roc_curve, roc_auc_score
import matplotlib.pyplot as plt
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

### Loading the Dataset, applying Padding and Splitting the data

In [ ]:
# load the dataset but only keep the top n words, zero the rest
top_words = 8000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

# pad dataset to a maximum review length in words
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

X = numpy.concatenate((X_train, X_test), axis=0)
y = numpy.concatenate((y_train, y_test), axis=0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) # split into train and test sets.

### Defining the Model Architecture

In [ ]:
model = Sequential()
model.add(Embedding(top_words, 100, input_length=max_words))
model.add(LSTM(100,return_sequences=True))
model.add(Conv1D(64, 3, padding='same',activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.8))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
visualizer(model, file_name="lstm_cnn", file_format="png", view=True)

In [ ]:
img = Image.open("lstm_cnn.png")
display(img)

### Training the Model


In [ ]:
# fit the model
history = model.fit(X_train, y_train, validation_split=0.2, epochs=3, batch_size=128, verbose=2)

# final evaluation of the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

model compiled
Instructions for updating:
Use tf.cast instead.
Train on 28000 samples, validate on 7000 samples
Epoch 1/3
 - 489s - loss: 0.4656 - acc: 0.7486 - val_loss: 0.2896 - val_acc: 0.8794
Epoch 2/3
 - 487s - loss: 0.2265 - acc: 0.9134 - val_loss: 0.3021 - val_acc: 0.8777
Epoch 3/3
 - 487s - loss: 0.1712 - acc: 0.9352 - val_loss: 0.2976 - val_acc: 0.8824
Accuracy: 88.49%


### Evaluating the Model

In [ ]:
pred = model.predict(X_test)
pred = [1 if x > 0.5 else 0 for x in pred]

##### Classification Report

In [ ]:
print('Classification Report:')
print(classification_report(y_test, pred))

##### Confusion Matrix

In [ ]:
print('Confusion Matrix:')
print(confusion_matrix(y_test, pred))

##### Evaluation Metrics

In [ ]:
precision = precision_score(y_test, pred)
recall = recall_score(y_test, pred)
f1 = f1_score(y_test, pred)
specificity = specificity_score(y_test, pred)
accuracy = accuracy_score(y_test, pred)

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'Specificity: {specificity}')
print(f'Accuracy: {accuracy}')

##### ROC Curve

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, predictions)
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

### Saving the Model

In [ ]:
model.save("lstm_cnnmodel.h5")